<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E6%9D%A5%E5%A0%B4%E8%80%85n%E5%B9%B4%E5%88%86%E3%82%B3%E3%82%A2%E5%95%86%E5%9C%8F%E3%83%9E%E3%83%83%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
# 郵便番号マッピング → 複数回開催 n 回分の統合コア商圏（○○位置.csv 版）
# -----------------------------------------------
# すること：
#  1) 各回ごとに「会場からの距離で近い順90％」の点集合を作る
#  2) n回分の90％点を全部まとめて：
#      - 郵便番号ごとの合計レコード数をカウント
#      - 「合計1レコードしかない郵便番号」を除外
#      - 残りを重複込みで数え直し、会場から遠い10％をカット
#        → 残り90％で concave hull を作り「統合コア商圏」にする
#  3) 各回の90％面＋統合コア面＋会場ピン＋点群を1つのKMLに出力
#
# 必要ファイル（Colab にアップロードしてから実行）：
#   - utf_ken_all.csv
#   - ○○位置.csv （例: 福岡位置.csv, 愛知位置.csv ... 複数アップロード可）
#       列: 「都道府県名」「市区町村名」「大字町丁目名」「緯度」「経度」
#   - 過去 n 回ぶんの来場CSV : A列に郵便番号（B列以降はそのまま持つ）
# ===============================================

!pip install -q simplekml shapely alphashape

# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV_LIST = [
    "41-10-2wハイブ長岡新規対象来場237／626.csv",
     "42-7-4wハイブ長岡新規対象来場357／769.csv",
     "AJ41-5-2wハイブ長岡183／446.csv",
    # ...
]
VENUE_ZIP   = "950-0078"          # 会場の郵便番号（ハイフンあり/なしどちらでもOK）
OUTPUT_KML  = "ハイブ長岡_90pct.kml"  # 出力KMLファイル名

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob
import numpy as np
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint

# ---------- 共通関数 ----------

def read_csv_auto(path, header_opt):
    """
    cp932 → utf-8-sig → utf-8 の順で読み込みを試す簡易関数
    header_opt: None -> header=None, 0 -> header=0
    """
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    """
    町名の「（…）」や「一丁目〜九丁目／1丁目〜9丁目」などを落としてベース名を作る
    例:
      上津台一丁目 → 上津台
      大通西（１〜１９丁目） → 大通西
    """
    t = str(t)
    t = re.sub(r"（.*?）", "", t)                     # （〜）を削除
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)  # ○丁目を削除
    return t.strip()

def build_alpha_shape(coords, target_ratio=0.1):
    """
    点集合 coords ([(lon, lat), ...]) から concave hull を作る。
    - convex hull 面積に対する比率 target_ratio に近い α をスキャン
    - 線や点になった場合は薄く buffer して面に変換
    - 失敗した場合は凸包にフォールバック
    """
    if len(coords) < 3:
        return None

    mp_all = MultiPoint(coords)
    convex = mp_all.convex_hull
    convex_area = convex.area
    if convex_area == 0:
        poly = convex
    else:
        alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10
        best_alpha = None
        best_shape = None
        best_diff = float("inf")

        for a in alpha_candidates:
            try:
                shp = alphashape.alphashape(coords, a)
            except Exception:
                continue
            if shp.is_empty or shp.area == 0:
                continue

            area = shp.area
            ratio = area / convex_area
            diff = abs(ratio - target_ratio)
            if diff < best_diff:
                best_diff = diff
                best_alpha = a
                best_shape = shp

        print("  chosen alpha:", best_alpha, " diff_from_target=", best_diff)
        poly = best_shape if best_shape is not None else convex

    # ポリゴン以外（線・点など）の場合は、薄く buffer して面にする
    if poly.geom_type in ("LineString", "MultiLineString", "Point", "MultiPoint"):
        poly = poly.buffer(1e-4)

    return poly

# ---------- 位置情報マスタの準備（utf_ken_all + ○○位置.csv） ----------

# utf_ken_all 読み込み
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)

if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

# ○○位置.csv を全部読み込んで全国マスタ作成
loc_list = []
for path in glob.glob("*位置*.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    required = {"都道府県名", "市区町村名", "大字町丁目名", "緯度", "経度"}
    if not required.issubset(loc.columns):
        print(f"  {path} は必須列が足りないのでスキップ: {loc.columns}")
        continue

    df_pos = loc.copy()
    df_pos = df_pos.rename(columns={
        "都道府県名": "pref",
        "市区町村名": "city",
        "大字町丁目名": "town",
        "緯度": "lat",
        "経度": "lon",
    })

    for c in ["pref", "city", "town"]:
        df_pos[c] = df_pos[c].astype(str).str.strip()
    df_pos["town_base"] = df_pos["town"].map(make_town_base)

    loc_list.append(df_pos[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("『○○位置.csv』が見つからないか、必須列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])

# 町名ベースごとに重心（平均）をとる
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# ---------- 会場の緯度経度を VENUE_ZIP から取得 ----------

venue_zip7 = re.sub(r"\D", "", VENUE_ZIP).zfill(7)

venue_addr = utf_use[utf_use["zip7"] == venue_zip7].copy()
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} が utf_ken_all.csv に見つかりません。")

venue_addr["town_base"] = venue_addr["town"].map(make_town_base)
venue_addr = venue_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
venue_addr = venue_addr.dropna(subset=["lat", "lon"])

if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} に対応する緯度経度が ○○位置.csv から取得できません。")

venue_lat = float(venue_addr.iloc[0]["lat"])
venue_lon = float(venue_addr.iloc[0]["lon"])
venue_point = Point(venue_lon, venue_lat)

print(f"会場座標: zip={venue_zip7}, lat={venue_lat}, lon={venue_lon}")

# ---------- 各回の90％商圏を計算 ----------

all_event_90_list = []   # 各回の90％点 DataFrame
event_polygons = []      # 各回の90％面ポリゴン

for idx, csv_path in enumerate(VISITOR_CSV_LIST, start=1):
    print(f"\n=== イベント{idx}: {csv_path} を処理中 ===")
    vis = read_csv_auto(csv_path, header_opt=None)
    vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

    # A列 → 郵便番号
    vis["zip7"] = (
        vis["col_0"]
        .astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(7)
    )
    vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
    if vis.empty:
        print("  有効な郵便番号が1件もないためスキップ")
        continue

    # 郵便番号 → pref/city/town
    vis_addr = vis.merge(utf_use, on="zip7", how="left")
    if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
        print("  utf_ken_all にマッチする郵便番号がないためスキップ")
        continue

    # 緯度経度付与
    vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
    vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
    vis_geo = vis_geo.dropna(subset=["lat", "lon"]).copy()
    if vis_geo.empty:
        print("  緯度経度が1件も付かなかったためスキップ")
        continue

    vis_geo["lat"] = vis_geo["lat"].astype(float)
    vis_geo["lon"] = vis_geo["lon"].astype(float)

    # 会場からの距離（度単位・順位付け用）
    vis_geo["geom"] = [
        Point(lo, la) for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
    ]
    vis_geo["dist"] = vis_geo["geom"].apply(lambda p: p.distance(venue_point))

    # 近い順に並べて上位90％を残す
    vis_geo_sorted = vis_geo.sort_values("dist").reset_index(drop=True)
    n = len(vis_geo_sorted)
    keep_n = math.ceil(n * 0.9)
    vis_90 = vis_geo_sorted.iloc[:keep_n].copy()
    vis_90["event_id"] = idx

    print(f"  総レコード数 {n} → 90％残し {keep_n} 件")

    # この回の90％点で concave hull を作る
    coords = list(zip(vis_90["lon"], vis_90["lat"]))
    print("  concave hull 計算中（イベントごと）...")
    hull = build_alpha_shape(coords, target_ratio=0.1)
    event_polygons.append((idx, hull))

    all_event_90_list.append(vis_90)

if not all_event_90_list:
    raise ValueError("有効なイベントデータが1件もありません。VISITOR_CSV_LIST とファイル内容を確認してください。")

# ---------- n回分90％データを統合 → 1人しかいない郵便番号を除外 ----------

all_90 = pd.concat(all_event_90_list, ignore_index=True)

# 郵便番号ごとの合計レコード数
zip_counts = all_90.groupby("zip7")["zip7"].transform("size")

# 合計1レコードしかない郵便番号を除外
mask_multi = zip_counts >= 2
removed_single = (~mask_multi).sum()
core_base = all_90[mask_multi].copy()

print(f"\n統合90％データ総数: {len(all_90)}")
print(f"『n回合計で1レコードしかない郵便番号』除外数: {removed_single}")
print(f"残りレコード数: {len(core_base)}")

if core_base.empty:
    raise ValueError("1回しか出てこない郵便番号を除いた結果、データが空になりました。")

# ---------- 統合データから遠い10％をカット ----------

core_base_sorted = core_base.sort_values("dist").reset_index(drop=True)
N_core = len(core_base_sorted)
keep_core_n = math.ceil(N_core * 0.9)  # 近い90％だけ残す
core_final = core_base_sorted.iloc[:keep_core_n].copy()

print(f"統合ベース {N_core} → 遠い10％カット後 {keep_core_n} 件")

# concave hull（統合コア商圏）
core_coords = list(zip(core_final["lon"], core_final["lat"]))
print("統合コア商圏 concave hull 計算中...")
core_hull = build_alpha_shape(core_coords, target_ratio=0.1)

# ---------- KML 出力（各回90％面＋統合コア面＋会場ピン＋点群） ----------

kml = simplekml.Kml()

# 会場マーカー
venue_pt = kml.newpoint(
    name=f"Venue_{venue_zip7}",
    coords=[(venue_lon, venue_lat)],
)
venue_pt.style.iconstyle.scale = 1.2
venue_pt.style.iconstyle.color = simplekml.Color.red

# 各回の90％面（薄めの色）
for idx, hull in event_polygons:
    if hull is None or hull.is_empty:
        continue

    # MultiPolygon の場合はいちばん大きい面
    if hull.geom_type == "MultiPolygon":
        hull = max(list(hull.geoms), key=lambda g: g.area)
    # 念のためポリゴン以外ならbufferで面化
    if hull.geom_type != "Polygon":
        hull = hull.buffer(1e-4)

    coords = list(hull.exterior.coords)

    poly = kml.newpolygon(
        name=f"event_{idx}_90pct_area",
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly.style.polystyle.color = simplekml.Color.changealpha('3f', simplekml.Color.blue)
    poly.style.polystyle.fill = 1
    poly.style.polystyle.outline = 1

# 各回90％点（フォルダごと）
for idx, vis_90 in enumerate(all_event_90_list, start=1):
    folder_pts = kml.newfolder(name=f"event_{idx}_90pct_points")
    for _, row in vis_90.iterrows():
        p = folder_pts.newpoint(
            coords=[(row["lon"], row["lat"])],
        )
        p.name = str(row.get("zip7", ""))
        p.style.iconstyle.scale = 0.5
        p.style.iconstyle.color = simplekml.Color.white

# 統合コア商圏（濃い紫）
if core_hull is not None and not core_hull.is_empty:
    if core_hull.geom_type == "MultiPolygon":
        core_hull = max(list(core_hull.geoms), key=lambda g: g.area)
    if core_hull.geom_type != "Polygon":
        core_hull = core_hull.buffer(1e-4)

    coords = list(core_hull.exterior.coords)
    poly_core = kml.newpolygon(
        name="core_merged_90pct_area",
        outerboundaryis=[(lon, lat) for lon, lat in coords],
    )
    poly_core.style.polystyle.color = simplekml.Color.changealpha('7f', simplekml.Color.purple)
    poly_core.style.polystyle.fill = 1
    poly_core.style.polystyle.outline = 1

# 統合コア点群
folder_core_pts = kml.newfolder(name="core_merged_90pct_points")
for _, row in core_final.iterrows():
    p = folder_core_pts.newpoint(
        coords=[(row["lon"], row["lat"])],
    )
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.yellow

kml.save(OUTPUT_KML)
print(f"\n完了：{OUTPUT_KML} を出力しました。")
print("  ・各回の90％商圏（event_1_90pct_area, event_2_90pct_area, ...）")
print("  ・n回統合コア商圏（core_merged_90pct_area）")


utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 新潟位置.csv
新潟位置.csv を encoding='cp932' で読み込み成功
会場座標: zip=9500078, lat=37.928232, lon=139.061913

=== イベント1: 41-10-2wハイブ長岡新規対象来場237／626.csv を処理中 ===
41-10-2wハイブ長岡新規対象来場237／626.csv を encoding='cp932' で読み込み成功
  総レコード数 208 → 90％残し 188 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.1907222294259365

=== イベント2: 42-7-4wハイブ長岡新規対象来場357／769.csv を処理中 ===
42-7-4wハイブ長岡新規対象来場357／769.csv を encoding='cp932' で読み込み成功
  総レコード数 319 → 90％残し 288 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.5647113838357679

=== イベント3: AJ41-5-2wハイブ長岡183／446.csv を処理中 ===
AJ41-5-2wハイブ長岡183／446.csv を encoding='cp932' で読み込み成功
  総レコード数 165 → 90％残し 149 件
  concave hull 計算中（イベントごと）...
  chosen alpha: 10.0  diff_from_target= 0.16486118125167534

統合90％データ総数: 625
『n回合計で1レコードしかない郵便番号』除外数: 258
残りレコード数: 367
統合ベース 367 → 遠い10％カット後 331 件
統合コア商圏 concave hull 計算中...
  chosen alpha: 10.0  diff_from_target= 0.24728055410324926

完了：ハイ